In [1]:
import sainomore
import torch
from sainomore import Elissabeth
import fruits
import math
import matplotlib.pyplot as plt

$$\newcommand{\iss}{\operatorname{ISS}}$$

In [2]:
config = {
    "input_vocab_size": 1,
    "input_type": "vector",
    "context_length": 125,
    "d_hidden": 1,
    "layer_norm": False,

    "lengths": [1, 2, 3, 4, 5, 6],
    "d_values": 1,
    "v_norm": False,
    "sum_normalization": False,
}

In [3]:
model = Elissabeth.build(config)
model.set_eye("embedding.weight")
for i in range(len(model.layers[0].levels)):
    torch.nn.init.ones_(model.get_parameter(f"layers.0.levels.{i}.P_V.transform.weight"))
    torch.nn.init.zeros_(model.get_parameter(f"layers.0.levels.{i}.P_V.transform.bias"))
torch.nn.init.ones_(model.get_parameter("layers.0.W_H"))
model.set_eye("layers.0.W_O")
model.set_eye("unembedding.weight")
watcher = sainomore.xai.ElissabethWatcher(model)

In [4]:
def iss(target, level):
    return model.layers[0].levels[level](target.unsqueeze(0).unsqueeze(-1)).detach().squeeze()[..., -1]


$$
\begin{align*}
\iss(x)^{[1][1][1][1][1]}_{0,T}=\Big(
    &\iss(x)^{[1][1][1][1][1]}_{0,s}+\\
    &\iss(x)^{[1][1][1][1]}_{0,s}\cdot \iss(x)^{[1]}_{s,T}+\\
    &\iss(x)^{[1][1][1]}_{0,s}\cdot \iss(x)^{[1][1]}_{s,T}+\\
    &\iss(x)^{[1][1]}_{0,s}\cdot \iss(x)^{[1][1][1]}_{s,T}+\\
    &\iss(x)^{[1]}_{0,s}\cdot \iss(x)^{[1][1][1][1]}_{s,T}+\\
    &\iss(x)^{[1][1][1][1][1]}_{s,T}\Big)
\end{align*}
$$

```python
d_hidden=6
lengths=[1,2,3,4,5]
```
use two $P_v=W_2\operatorname{ReLU}(W_1 u + b_1)+b_2$ value projections, one with (weights for time
component, identity else)
$W_2=1,W_1=1,b_1=s,b_2=0$ 

In [5]:
x = torch.randn((125,))

In [7]:
approx = (
      iss(x[:s+1], 4)
    + iss(x[:s+1], 3)*iss(x[s+1:], 0)
    + iss(x[:s+1], 2)*iss(x[s+1:], 1)
    + iss(x[:s+1], 1)*iss(x[s+1:], 2)
    + iss(x[:s+1], 0)*iss(x[s+1:], 3)
    +                 iss(x[s+1:], 4)
)

In [8]:
approx, iss(x, 4)

(tensor(8730.6875), tensor(8730.6865))